<a href="https://colab.research.google.com/github/sbbwoy/ESAA/blob/main/220624_cifar10_with_CNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2. 전이학습(transfer learning)을 통한 이미지 분류

In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import random
import numpy as np
from tqdm import tqdm

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from multiprocessing import cpu_count
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.nn import CrossEntropyLoss
from torchvision.models import efficientnet_b3 as efficientnet
from sklearn.model_selection import train_test_split

### 데이터 load, 전처리

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(), #이미지 데이터를 tensor 데이터 포멧으로 바꾸어줍니다.
    transforms.Resize([224,224]), #이미지의 크기가 다를 수 있으니 크기를 통일해 줍니다.
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) #픽셀 단위 데이터를 정규화 시켜줍니다.
])

In [4]:
import torchvision

train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [5]:
train_idx, valid_idx = train_test_split(np.arange(len(train_data)), test_size=0.2, 
                                        random_state=42, shuffle=True, stratify=train_data.targets)



*   batch_size : 하드웨어에 한번에 load할 데이터의 크기
* num_workers : 데이터 로드 멀티 프로세싱을 위한 파라미터


In [6]:
batch_size = 32
num_workers = int(cpu_count() / 2)

- data loader : 데이터 셋을 순회하며 모델에 데이터를 넣어주는 객체

In [7]:
train_loader = DataLoader(train_data, batch_size=batch_size, 
                          sampler=SubsetRandomSampler(train_idx), num_workers=num_workers)
valid_loader = DataLoader(train_data, batch_size=batch_size, 
                          sampler=SubsetRandomSampler(valid_idx), num_workers=num_workers)

데이터 크기 확인

In [8]:
train_total = len(train_idx)
valid_total = len(valid_idx)

train_batches = len(train_loader)
valid_batches = len(valid_loader)

In [9]:
print('total train imgs :',train_total,'/ total train batches :', train_batches)
print('total valid imgs :',valid_total, '/ total valid batches :', valid_batches)

total train imgs : 40000 / total train batches : 1250
total valid imgs : 10000 / total valid batches : 313


## 모델 불러오기 / 파라미터 설정
전이학습을 위한 모델 load

- Pytorch의 models 메소드 사용하여 외부 모델 불러오기
    - efficientnet_b3 모델 사용
        - pretrained = True : ImageNet이라는 데이터셋을 대상으로 학습된 모델이 load
        - *pretrained* = False : 모델의 구조만 load되고 모델의 가중치들은 load 되지 않음

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.is_available()

True

In [10]:
net = models.efficientnet_b3(pretrained=False) # 사전 학습 모델 가중치 사용 안함
net.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)

- 모델에 데이터를 학습하기 위해 마지막 layer의 output size와 분류할 라벨 수 입력

In [13]:
net.fc = nn.Linear(1000, 10)
net = net.to(device)

In [14]:
criterion = CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
epochs = 10

## 학습

In [15]:
for epoch in range(epochs):
    net.train()
    
    train_loss = 0 
    train_correct = 0
    tqdm_dataset = tqdm(train_loader)
    for x,y in tqdm_dataset:
        x = x.to(device)
        y = y.to(device)
        outputs = net(x)
        loss = criterion(outputs,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(y).sum().item()
        
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(loss.item()),
        })

    train_loss = train_loss / train_batches
    train_acc = train_correct / train_total
    
    net.eval()
    
    valid_loss = 0 
    valid_correct = 0
    tqdm_dataset = tqdm(valid_loader)
    with torch.no_grad():
        for x,y in tqdm_dataset:
            x = x.to(device)
            y = y.to(device)
            
            outputs = net(x)
            loss = criterion(outputs, y)
            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            valid_correct += predicted.eq(y).sum().item()
            
            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(loss.item()),
            })
            
    valid_loss = valid_loss / valid_batches
    valid_acc = valid_correct / valid_total
        
    print('epochs',epoch+1, 'train loss',train_loss,'train acc', train_acc, 'valid loss',valid_loss, 'valid acc',valid_acc)

100%|██████████| 313/313 [00:29<00:00, 10.69it/s, Epoch=1, Loss=1.373059]


epochs 1 train loss 1.761773354625702 train acc 0.3551 valid loss 1.3566520785371337 valid acc 0.521


100%|██████████| 313/313 [00:29<00:00, 10.57it/s, Epoch=2, Loss=1.473063]


epochs 2 train loss 1.1705012971878053 train acc 0.582475 valid loss 0.9914081308026664 valid acc 0.6445


100%|██████████| 313/313 [00:29<00:00, 10.70it/s, Epoch=3, Loss=1.251852]


...

epochs 8 train loss 0.36826473402380944 train acc 0.874725 valid loss 0.40336948890274704 valid acc 0.8662


100%|██████████| 313/313 [00:29<00:00, 10.69it/s, Epoch=9, Loss=1.173254]


epochs 9 train loss 0.3125938187122345 train acc 0.892175 valid loss 0.40120087408076843 valid acc 0.8687


100%|██████████| 313/313 [00:29<00:00, 10.73it/s, Epoch=10, Loss=0.515993]

epochs 10 train loss 0.27538062208294867 train acc 0.905075 valid loss 0.42288211708108836 valid acc 0.8623


## 모델 저장 / 불러오기

학습된 모델의 가중치 저장

In [16]:
path = './model.pth'
torch.save(net.state_dict(),path)

저장된 모델 불러오기

In [17]:
path = './model.pth'
net.load_state_dict(torch.load(path))

<All keys matched successfully>

## 추론

In [18]:
from glob import glob
import PIL.Image
import numpy as np

test_images = []

path = './data/'
for filename in sorted(glob(path + "test/*.jpg")):
    an_img = PIL.Image.open(filename) 
    img_array = np.array(an_img) 
    test_images.append(img_array) 

test_images = np.array(test_images)

In [19]:
class CustomDataset(Dataset):
    def __init__(self, transform):
        self.transform = transform 
        self.img_list = test_images
        self.img_labels = [0] * 10000 

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        return self.transform(self.img_list[idx]), self.img_labels[idx]

In [20]:
test_set = CustomDataset(transform)

In [21]:
test_loader = DataLoader(test_set, batch_size = batch_size, num_workers=num_workers)

- test 데이터 예측

    - 학습이 진행되지 않도록 net.eval() 코드 작성 : 데이터가 back propagation 되어 가중치가 수정되지 않도록 함
    - 현재 sample_submission.csv 파일을 다운로드할 수 없어 아래 코드 실행은 생략

In [22]:
import pandas as pd
sample_submission = pd.read_csv('/content/sample_submission.csv')

net.eval()

batch_index = 0

for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    outputs = net(images)
    batch_index = i * batch_size
    max_vals, max_indices = torch.max(outputs, 1)
    sample_submission.iloc[batch_index:batch_index + batch_size, 1:] = max_indices.long().cpu().numpy()[:,np.newaxis]

FileNotFoundError: ignored

- 라벨 복원: 예측된 데이터의 라벨이 숫자로 되어 있기 때문에 라벨 복원 필요

In [ ]:
labels = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer',
          5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
sample_submission['target'] = sample_submission['target'].map(labels)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('submit.csv',index=False)